In [66]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [82]:
num_epochs = 15
batch_size = 32
lr = 0.001
wd = 0

classes = ['healthy', 'non_healthy']

In [68]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_dir = '../data/'

In [69]:
mean = [0.44947562, 0.46524084, 0.40037745]
std = [0.18456618, 0.16353698, 0.20014246]

data_transforms = {
        'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)])}

test_images = datasets.ImageFolder(os.path.join(data_dir, 'test'),
                    data_transforms['test'])

test_dataloader = DataLoader(test_images, batch_size=batch_size, shuffle=True, num_workers=4)

In [75]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # 3 input image channel,
        # 6 output channel,
        # 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation
        self.fc1 = nn.Linear(16 * 61 * 61, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)
        
        nn.init.xavier_normal_(self.conv1.weight)
        nn.init.xavier_normal_(self.conv2.weight)
        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)
        nn.init.xavier_normal_(self.fc3.weight)
        
    def forward(self, x):
        # max pooling over a (2, 2) windows
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [76]:
model = Model()
model.load_state_dict(torch.load('model', map_location=str(device)))
model.eval()

Model(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=59536, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=1, bias=True)
)

In [77]:
class_corrects = [0] * len(classes)
class_total = [0] * len(classes)
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.round(F.sigmoid(outputs))
        c = (preds.view(-1) == labels.data.float()).squeeze()
        for i in range(len(c)):
            label = labels[i]
            class_corrects[label] += c[i].item()
            class_total[label] += 1        

In [78]:
def print_to_file(to_file, filename='model_evaluation'):
    f = open(filename, 'a')
    f.write(to_file)
    f.close()

In [81]:
for i in range(len(classes)):
    print_to_file('Accuracy of class "{}" = {}\n'.format(classes[i], class_corrects[i] / class_total[i]))

conf_matrix = {'TP': class_corrects[0], 'FN': class_total[0] - class_corrects[0],
                    'FP': class_total[1] - class_corrects[1], 'TN': class_corrects[1]}

print_to_file('\nPositive class = "{}"'.format(classes[0]))
print_to_file('\nNegative class = "{}"'.format(classes[1]))

print_to_file('\n\nConfusion matrix: ')
print_to_file('\nTP = {}  ||  FN = {}'.format(conf_matrix['TP'], conf_matrix['FN']))
print_to_file('\nFP = {}  ||  TN = {}'.format(conf_matrix['FP'], conf_matrix['TN']))

print_to_file('\n\nAccuracy = {}'.format((conf_matrix['TP'] + conf_matrix['TN']) / 
                             (conf_matrix['TP'] + conf_matrix['TN'] + conf_matrix['FP'] + conf_matrix['FN'])))
print_to_file('\nPrecision = {}'.format(conf_matrix['TP'] / (conf_matrix['TP'] + conf_matrix['FP'])))
print_to_file('\nRecall = {}'.format(conf_matrix['TP'] / (conf_matrix['TP'] + conf_matrix['FN'])))
print_to_file('\nF1 - measure = {}'. format(2 * conf_matrix['TP'] / 
                                            (2 * conf_matrix['TP'] + conf_matrix['FN'] + conf_matrix['FP'])))

print_to_file('\n\nTrue Positive Rate = {}'.format(conf_matrix['TP'] / (conf_matrix['TP'] + conf_matrix['FN'])))
print_to_file('\nFalse Positive Rate = {}'.format(conf_matrix['FP'] / (conf_matrix['TN'] + conf_matrix['FP'])))

print('Analysis measures saved to file.')